In [10]:
import rdflib
import csv
import time
import pandas as pd

ruta_csv = "../bbdd/datos_ensembl" 
ncrna = rdflib.Graph()
ncrna.parse("ncrna.txt", format="turtle")

<Graph identifier=N0f2ff8bac01848f9ade2618cad39747a (<class 'rdflib.graph.Graph'>)>

In [11]:
start = time.time()

# enlazar prefijos a namespaces
ncrna.bind("rdf", rdflib.URIRef("http://www.w3.org/1999/02/22-rdf-syntax-ns#"))
ncrna.bind("owl", rdflib.URIRef("http://www.w3.org/2002/07/owl#"))
ncrna.bind("rdfs", rdflib.URIRef("http://www.w3.org/2000/01/rdf-schema#"))
ncrna.bind("biolink", rdflib.URIRef("https://wx3id.org/biolink/vocab/"))
ncrna.bind("skos", rdflib.URIRef("http://www.w3.org/2004/02/skos/core#"))
ncrna.bind("dcterms", rdflib.URIRef("http://purl.org/dc/terms/"))
ncrna.bind("obo", rdflib.URIRef("http://purl.obolibrary.org/obo/"))
ncrna.bind("sio", rdflib.URIRef("http://semanticscience.org/resource/"))
ncrna.bind("faldo", rdflib.URIRef("http://biohackathon.org/resource/faldo#"))
ncrna.bind("ncrnagenebg", rdflib.URIRef("http://rdf.biogateway.eu/ncrnagene/9606/"))
ncrna.bind("genebg", rdflib.URIRef("http://rdf.biogateway.eu/gene/9606/"))
ncrna.bind("ncrnabg", rdflib.URIRef("http://rdf.biogateway.eu/ncrna/9606/"))
ncrna.bind("assembly", rdflib.URIRef("https://www.ncbi.nlm.nih.gov/assembly/"))
ncrna.bind("schema", rdflib.URIRef("http://schema.org/"))
ncrna.bind("nuccore", rdflib.URIRef("https://www.ncbi.nlm.nih.gov/nuccore/"))
ncrna.bind("ncbipubmed", rdflib.URIRef("https://www.ncbi.nlm.nih.gov/pubmed/"))
ncrna.bind("idpubmed", rdflib.URIRef("http://identifiers.org/pubmed/"))
ncrna.bind("omim", rdflib.URIRef("http://purl.bioontology.org/ontology/OMIM/"))
ncrna.bind("bao", rdflib.URIRef("http://www.bioassayontology.org/bao#"))
ncrna.bind("rnacentral", rdflib.URIRef("https://rnacentral.org/"))
ncrna.bind("ensembl", rdflib.URIRef("https://www.ensembl.org/Homo_sapiens/Gene/Summary?db=core;g="))


# definir un diccionario con los ID de las bases de datos que se usen
database_ids = {
    "RefSeq": "http://purl.obolibrary.org/obo/NCIT_C45335",  
    "Ensembl": "http://purl.obolibrary.org/obo/NCIT_C45763",
    "miRBase": "http://purl.obolibrary.org/obo/MI_2358",
    "miRCancer": "http://mircancer.ecu.edu/",
    "LNCipedia": "https://lncipedia.org/download"
}

# definir diccionarios para mapear biotipos
gene_biotype = {
    "lncRNA": "http://purl.obolibrary.org/obo/SO_0002127",
    "miRNA": "http://purl.obolibrary.org/obo/SO_0001265",
    "Mt_rRNA": "http://purl.obolibrary.org/obo/SO_0002363",
    "ribozyme": "http://purl.obolibrary.org/obo/SO_0002181",
    "rRNA": "http://purl.obolibrary.org/obo/SO_0001637",
    "scaRNA": "http://purl.obolibrary.org/obo/SO_0002339",
    "scRNA": "http://purl.obolibrary.org/obo/SO_0001266",
    "snoRNA": "http://purl.obolibrary.org/obo/SO_0001267",
    "snRNA": "http://purl.obolibrary.org/obo/SO_0001268",
    "miscRNA": "http://purl.obolibrary.org/obo/OGG_0000000032",
}

transcript_biotype = {
    "lncRNA": "http://purl.obolibrary.org/obo/SO_0001877",
    "miRNA": "http://purl.obolibrary.org/obo/SO_0000276",
    "Mt_rRNA": "http://purl.obolibrary.org/obo/SO_0002128",
    "Mt_tRNA": "http://purl.obolibrary.org/obo/SO_0002129",
    "retained_intron": "http://purl.obolibrary.org/obo/SO_0002113",
    "ribozyme": "http://purl.obolibrary.org/obo/SO_0000374",
    "rRNA": "http://purl.obolibrary.org/obo/SO_0000252",
    "scaRNA": "http://purl.obolibrary.org/obo/SO_0002095",
    "scRNA": "http://purl.obolibrary.org/obo/SO_0000013",
    "snoRNA": "http://purl.obolibrary.org/obo/SO_0000275",
    "snRNA": "http://purl.obolibrary.org/obo/SO_0000274",
    "sncRNA": "http://purl.obolibrary.org/obo/SO_0002247",
    "sRNA": "http://purl.obolibrary.org/obo/SO_0002182",
    "TEC": "http://purl.obolibrary.org/obo/SO_0002139",
    "vault_RNA": "http://purl.obolibrary.org/obo/SO_0000404"
}

chromosome_uris = {
    "chr1": "https://www.ncbi.nlm.nih.gov/nuccore/NC_000001.11",
    "chr2": "https://www.ncbi.nlm.nih.gov/nuccore/NC_000002.12",
    "chr3": "https://www.ncbi.nlm.nih.gov/nuccore/NC_000003.12",
    "chr4": "https://www.ncbi.nlm.nih.gov/nuccore/NC_000004.12",
    "chr5": "https://www.ncbi.nlm.nih.gov/nuccore/NC_000005.10",
    "chr6": "https://www.ncbi.nlm.nih.gov/nuccore/NC_000006.12",
    "chr7": "https://www.ncbi.nlm.nih.gov/nuccore/NC_000007.14",
    "chr8": "https://www.ncbi.nlm.nih.gov/nuccore/NC_000008.11",
    "chr9": "https://www.ncbi.nlm.nih.gov/nuccore/NC_000009.12",
    "chr10": "https://www.ncbi.nlm.nih.gov/nuccore/NC_000010.11",
    "chr11": "https://www.ncbi.nlm.nih.gov/nuccore/NC_000011.10",
    "chr12": "https://www.ncbi.nlm.nih.gov/nuccore/NC_000012.12",
    "chr13": "https://www.ncbi.nlm.nih.gov/nuccore/NC_000013.11",
    "chr14": "https://www.ncbi.nlm.nih.gov/nuccore/NC_000014.9",
    "chr15": "https://www.ncbi.nlm.nih.gov/nuccore/NC_000015.10",
    "chr16": "https://www.ncbi.nlm.nih.gov/nuccore/NC_000016.10",
    "chr17": "https://www.ncbi.nlm.nih.gov/nuccore/NC_000017.11",
    "chr18": "https://www.ncbi.nlm.nih.gov/nuccore/NC_000018.10",
    "chr19": "https://www.ncbi.nlm.nih.gov/nuccore/NC_000019.10",
    "chr20": "https://www.ncbi.nlm.nih.gov/nuccore/NC_000020.11",
    "chr21": "https://www.ncbi.nlm.nih.gov/nuccore/NC_000021.9",
    "chr22": "https://www.ncbi.nlm.nih.gov/nuccore/NC_000022.11",
    "chrX": "https://www.ncbi.nlm.nih.gov/nuccore/NC_000023.11",
    "chrY": "https://www.ncbi.nlm.nih.gov/nuccore/NC_000024.10",
    "chrM": "https://www.ncbi.nlm.nih.gov/nuccore/NC_012920.1"
}


def is_valid_int(value):
    try:
        int(value)
        return True
    except ValueError:
        return False


with open(ruta_csv, newline='', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        if row['Gene_ID'] in [None, '', 'NA']:
            continue
        # extraer ID de gen y transcrito si viene en formato de URI
        gene_id = row['Gene_ID'].split('/')[-1]  
        ncrna_id = row['ncRNA_ID'].split('/')[-1]  

        # indicar la base de datos de la que se obtiene la información
        data_source = "Ensembl"

        ncRNA_gene = rdflib.URIRef(f"http://rdf.biogateway.eu/ncrnagene/9606/{gene_id}#{data_source}")
        ncRNA_transcript = rdflib.URIRef(f"http://rdf.biogateway.eu/ncrna/9606/{ncrna_id}#{data_source}")


        # asignar el biotipo correspondiente del diccionario, si no se encuentra, se asigna el general
        database_uri = database_ids.get(data_source, "http://purl.obolibrary.org/obo/NCIT_C15426")


        # añadir las tripletas correspondientes
        ncrna.add((ncRNA_gene, rdflib.URIRef("http://www.w3.org/2004/02/skos/core#prefLabel"), rdflib.Literal(f"Gene encoding non-coding RNA ncRNA/{ncrna_id} according to {data_source}")))
        ncrna.add((ncRNA_gene, rdflib.URIRef("http://semanticscience.org/resource/SIO_010080"), ncRNA_transcript))
        ncrna.add((ncRNA_gene, rdflib.URIRef("http://purl.obolibrary.org/obo/RO_0002162"), rdflib.URIRef("http://purl.obolibrary.org/obo/NCBITaxon_9606")))
        ncrna.add((ncRNA_gene, rdflib.URIRef("http://purl.org/dc/terms/hasVersion"), rdflib.URIRef("https://www.ncbi.nlm.nih.gov/assembly/GCF_000001405")))
        
        
        if 'gncene_biotype' in row:
            gene_type = gene_biotype.get(row['gene_biotype'], "http://purl.obolibrary.org/obo/SO_0001263")
            ncrna.add((ncRNA_gene, rdflib.RDFS.subClassOf, rdflib.URIRef(gene_type)))
        if 'Chromosome' in row:
                chromosome_name = row['Chromosome']
                chr_uri = chromosome_uris.get(row['Chromosome'], "http://purl.obolibrary.org/obo/SO_0000340")
                ncrna.add((ncRNA_gene, rdflib.URIRef("http://purl.obolibrary.org/obo/BFO_0000050"), rdflib.URIRef(chr_uri)))
        if 'Start' in row and is_valid_int(row.get('Start')):
            start_position = row['Start']
            ncrna.add((ncRNA_gene, rdflib.URIRef("http://purl.obolibrary.org/obo/GENO_0000894"), rdflib.Literal(int(row['Start']), datatype=rdflib.XSD.integer)))       
        if 'End' in row and is_valid_int(row.get('End')):
            end_position = row['End']
            ncrna.add((ncRNA_gene, rdflib.URIRef("http://purl.obolibrary.org/obo/GENO_0000895"), rdflib.Literal(int(row['End']), datatype=rdflib.XSD.integer)))       
        if 'Strand' in row:
            if row['Strand'] in ['1', '-1']:
                strand_uri = rdflib.URIRef("http://biohackathon.org/resource/faldo#ForwardStrandPosition" if row['Strand'] == '1' else "http://biohackathon.org/resource/faldo#ReversePosition")
                ncrna.add((ncRNA_gene, rdflib.URIRef("http://purl.obolibrary.org/obo/Re"), strand_uri))
            else:
                print(f"Valor inesperado para Strand: {row['Strand']}")
        if 'End' and 'Start' and 'Chromosome' in row:
            gene_definition = f"Gene encoding non-coding RNA, located on chromosome {chromosome_name} of Homo sapiens, spanning from start position {start_position} to end position {end_position} according to {data_source}."
            ncrna.add((ncRNA_gene, rdflib.URIRef("http://www.w3.org/2004/02/skos/core#definition"), rdflib.Literal(gene_definition)))

        ncrna.add((ncRNA_transcript, rdflib.URIRef("http://www.w3.org/2004/02/skos/core#prefLabel"), rdflib.Literal(f"Non-coding RNA ncRNA/{ncrna_id} according to {data_source}")))      
        ncrna.add((ncRNA_transcript, rdflib.URIRef("http://semanticscience.org/resource/SIO_010081"), ncRNA_gene))
        ncrna.add((ncRNA_transcript, rdflib.URIRef("http://purl.obolibrary.org/obo/RO_0002162"), rdflib.URIRef("http://purl.obolibrary.org/obo/NCBITaxon_9606")))
        ncrna.add((ncRNA_transcript, rdflib.URIRef("http://schema.org/evidenceOrigin"), rdflib.URIRef(database_uri)))

        if 'transcript_biotype' in row:
            transcript_type = transcript_biotype.get(row['transcript_biotype'], "http://purl.obolibrary.org/obo/SO_0000655")
            ncrna.add((ncRNA_transcript, rdflib.RDFS.subClassOf, rdflib.URIRef(transcript_type)))
        if 'Description' in row and len(row['Description']) > 5:
            ncrna.add((ncRNA_transcript, rdflib.URIRef("http://www.w3.org/2004/02/skos/core#definition"), rdflib.Literal(row['Description'])))
        if 'transcript_id' in row:
            ncRNA_ensembl = row['transcript_id']
            ncRNA_ensembluri = rdflib.URIRef(f"https://www.ensembl.org/Homo_sapiens/Gene/Summary?db=core;g={ncRNA_ensembl}")    
            ncrna.add((ncRNA_transcript, rdflib.namespace.SKOS.exactMatch, rdflib.URIRef(ncRNA_ensembluri)))
        if 'ncRNA_name' in row:
            ncRNA_nameid = row['ncRNA_name']
            ncRNA_nameuri = rdflib.URIRef(f"https://rnacentral.org/{ncRNA_nameid}")           
            ncrna.add((ncRNA_transcript, rdflib.URIRef("http://www.w3.org/2002/07/owl#sameAs"), rdflib.URIRef(ncRNA_nameuri)))
        if 'OMIM_ID' in row and len(row['OMIM_ID']) > 3:
            omim_id = row['OMIM_ID']
            omim_uri = rdflib.URIRef(f"http://purl.bioontology.org/ontology/OMIM/{omim_id}")
            ncrna.add((ncRNA_transcript, rdflib.URIRef("http://purl.obolibrary.org/obo/RO_0003302"), rdflib.URIRef(omim_uri)))     
        if 'Pubmed_ID' in row:
            pubmed_id = row['Pubmed_ID']
            pubmed_ncbiuri = rdflib.URIRef(f"https://www.ncbi.nlm.nih.gov/pubmed/{pubmed_id}")
            pubmed_iduri = rdflib.URIRef(f"http://identifiers.org/pubmed/{pubmed_id}")
            ncrna.add((ncRNA_transcript, rdflib.URIRef("http://semanticscience.org/resource/SIO_000772"), rdflib.URIRef(pubmed_ncbiuri)))
            ncrna.add((rdflib.URIRef(pubmed_ncbiuri), rdflib.namespace.SKOS.exactMatch, rdflib.URIRef(pubmed_iduri) ))
            ncrna.add((rdflib.URIRef(pubmed_ncbiuri), rdflib.URIRef("https://wx3id.org/biolink/vocab/category"), rdflib.URIRef("https://wx3id.org/biolink/vocab/Article")))
        if 'Method' in row:
            ncrna.add((ncRNA_transcript, rdflib.URIRef("http://www.bioassayontology.org/bao#BAO_0000207"), rdflib.Literal(row['Method'])))
        if 'Expression' in row:
            if row['Expression'] in ['up', 'down']:
                exp_uri = rdflib.URIRef("http://purl.obolibrary.org/obo/MI_2235" if row['Expression'] == 'up' else "http://purl.obolibrary.org/obo/MI_2240")
                ncrna.add((ncRNA_transcript, exp_uri, rdflib.URIRef(omim_uri)))
            else:
                print(f"Valor inesperado para Expression: {row['Expression']}")

# guardar RDF en formato Turtle
output_name = f'ncrna_{data_source.lower()}.rdf'
ncrna.serialize(destination=output_name, format='turtle')

end = time.time()

In [12]:
total_time = (end - start) / 60
print(f"El fichero '{output_name}' ha sido creado en {total_time:.2f} minutos.")

El fichero 'ncrna_ensembl.rdf' ha sido creado en 0.78 minutos.
